In [1]:
# Создание наборов данных
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
import math

# Загрузка обучающего набора
data_train = pd.read_csv("dataset_train_yolo.csv")

# Загрузка тестового набора
data_test = pd.read_csv("dataset_test_yolo.csv")

In [2]:
# Вычисление угла между двумя прямыми
def angle(x0, y0, x1, y1, x2, y2):
    calc_angle = math.degrees(math.atan2(y2 - y0, x2 - x0) - math.atan2(y1 - y0, x1 - x0))
    if calc_angle < 0:
        calc_angle += 360
    return calc_angle

In [3]:
Angle_YOLO = ['RightElbowAngle', 'LeftElbowAngle', 'RightHandAngle', 'LeftHandAngle', 'RightKneeAngle', 'LeftKneeAngle',
                   'RightLegAngle', 'LeftLegAngle'
             ]
Key_Points_YOLO = ['Nose', 'LeftEye', 'RightEye', 'LeftEar', 'RightEar', 
                   'LeftShoulder', 'RightShoulder', 'LeftElbow', 'RightElbow', 
                   'LeftWrist', 'RightWrist', 'LeftHip', 'RightHip', 
                   'LeftKnee', 'RightKnee', 'LeftAnkle', 'RightAnkle'
                  ]

In [4]:
# Создание пустого датафрейма
def init_data(angle_name):
    data = []
    data.append("FileName")
    for p in angle_name:
        data.append(p)
    data.append("target")
    data = pd.DataFrame(columns = data) 
    return data

In [5]:
# Создание датасета каркасных моделей (набора углов и метки - названия позы)
def create_df_angle_pos(angle_name, df, name_file):
    data = init_data(angle_name)
    target = []
    for i in range(len(df)):
        temp = []
        temp += [df.loc[i]['FileName']]
        temp += [angle(df.loc[i]['RightElbow_x'], df.loc[i]['RightElbow_y'], 
                        df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'], 
                        df.loc[i]['RightWrist_x'], df.loc[i]['RightWrist_y'])]
        temp += [angle(df.loc[i]['LeftElbow_x'], df.loc[i]['LeftElbow_y'], 
                        df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'], 
                        df.loc[i]['LeftWrist_x'], df.loc[i]['LeftWrist_y'])] 
        temp += [angle(df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'],
                        df.loc[i]['RightElbow_x'], df.loc[i]['RightElbow_y'], 
                        df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'])]
        temp += [angle(df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'],
                        df.loc[i]['LeftElbow_x'], df.loc[i]['LeftElbow_y'], 
                        df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'])]
        temp += [angle(df.loc[i]['RightKnee_x'], df.loc[i]['RightKnee_y'],
                        df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'], 
                        df.loc[i]['RightAnkle_x'], df.loc[i]['RightAnkle_y'])]
        temp += [angle(df.loc[i]['LeftKnee_x'], df.loc[i]['LeftKnee_y'],
                        df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'], 
                        df.loc[i]['LeftAnkle_x'], df.loc[i]['LeftAnkle_y'])]
        temp += [angle(df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'],
                       df.loc[i]['RightKnee_x'], df.loc[i]['RightKnee_y'],
                       df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'])]
        temp += [angle(df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'],
                       df.loc[i]['LeftKnee_x'], df.loc[i]['LeftKnee_y'],
                       df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'])]
        temp += [df.loc[i]['target']]
        data.loc[i] = temp # Запись в датафрейм

    data.to_csv(name_file) # Запись датафрейма углов в файл
    return data

In [6]:
# Построение датафрейма угловых скелетных моделей обучающего набора
time_start = time.time()
data_tr_angle = create_df_angle_pos(Angle_YOLO, data_train, 'dataset_train_yolo_angle.csv')
time_end = time.time()
print('time: ', time_end-time_start)
print(data_tr_angle.info())
data_tr_angle.head()

time:  3.6398730278015137
<class 'pandas.core.frame.DataFrame'>
Index: 1081 entries, 0 to 1080
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FileName         1081 non-null   object 
 1   RightElbowAngle  1081 non-null   float64
 2   LeftElbowAngle   1081 non-null   float64
 3   RightHandAngle   1081 non-null   float64
 4   LeftHandAngle    1081 non-null   float64
 5   RightKneeAngle   1081 non-null   float64
 6   LeftKneeAngle    1081 non-null   float64
 7   RightLegAngle    1081 non-null   float64
 8   LeftLegAngle     1081 non-null   float64
 9   target           1081 non-null   object 
dtypes: float64(8), object(2)
memory usage: 92.9+ KB
None


,FileName,RightElbowAngle,LeftElbowAngle,RightHandAngle,LeftHandAngle,RightKneeAngle,LeftKneeAngle,RightLegAngle,LeftLegAngle,target
0,DATASET/TRAIN/goddess/00000112.jpg,254.719619,95.756570,278.093145,81.819444,164.562274,226.456000,141.613974,251.368236,goddess
1,DATASET/TRAIN/goddess/00000190.jpg,161.660203,203.508757,175.865468,171.231871,155.170690,171.758132,344.959159,346.793106,goddess
2,DATASET/TRAIN/goddess/00000348.jpg,164.929313,128.045148,343.511718,96.995172,306.509213,313.679921,29.378239,49.184764,goddess
3,DATASET/TRAIN/goddess/00000115.jpg,184.734978,155.849069,268.265609,93.856410,133.484720,218.593137,103.267217,251.093536,goddess
4,DATASET/TRAIN/goddess/00000358.jpg,229.345556,108.588067,278.313297,76.443765,171.685030,207.780170,152.415336,222.780967,goddess


In [7]:
# Построение датафрейма угловых скелетных моделей тестового набора
time_start = time.time()
data_te_angle = create_df_angle_pos(Angle_YOLO, data_test, 'dataset_test_yolo_angle.csv')
time_end = time.time()
print('time: ', time_end-time_start)
print(data_te_angle.info())
data_te_angle.head()

time:  1.5711002349853516
<class 'pandas.core.frame.DataFrame'>
Index: 470 entries, 0 to 469
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FileName         470 non-null    object 
 1   RightElbowAngle  470 non-null    float64
 2   LeftElbowAngle   470 non-null    float64
 3   RightHandAngle   470 non-null    float64
 4   LeftHandAngle    470 non-null    float64
 5   RightKneeAngle   470 non-null    float64
 6   LeftKneeAngle    470 non-null    float64
 7   RightLegAngle    470 non-null    float64
 8   LeftLegAngle     470 non-null    float64
 9   target           470 non-null    object 
dtypes: float64(8), object(2)
memory usage: 40.4+ KB
None


,FileName,RightElbowAngle,LeftElbowAngle,RightHandAngle,LeftHandAngle,RightKneeAngle,LeftKneeAngle,RightLegAngle,LeftLegAngle,target
0,DATASET/TEST/goddess/00000016.jpg,160.885343,185.576944,177.214806,177.329992,189.412789,175.199735,203.859499,160.711422,goddess
1,DATASET/TEST/goddess/00000057.jpg,209.860142,159.555976,298.897340,80.349340,174.761894,200.037193,144.343021,239.793261,goddess
2,DATASET/TEST/goddess/00000036.jpg,259.127096,88.282096,282.023145,75.400107,157.921006,230.761544,128.540709,252.643044,goddess
3,DATASET/TEST/goddess/00000088.jpg,241.129174,103.711100,275.062032,77.725315,147.528463,213.500616,130.670094,237.603980,goddess
4,DATASET/TEST/goddess/00000050.jpg,245.206824,109.379621,270.826504,89.862478,166.866823,217.437081,141.953858,235.008799,goddess
